In [34]:
import json
import pandas as pd
import folium
from matplotlib import colormaps
from matplotlib.colors import rgb2hex
import csv


In [36]:
with open('data/booking_hotels.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Grouper les données par ville
grouped_data = {}
for item in data:
    city = item['city']
    if city not in grouped_data:
        grouped_data[city] = []
    grouped_data[city].append(item)

csv_filename = 'data/best_cities_hotels.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    # Définir les en-têtes du fichier CSV
    fieldnames = ['city', 'hotel_name', 'hotel_address', 'latitude', 'longitude', 'hotel_rating', 'hotel_description', 'hotel_url']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Écrire les en-têtes dans le fichier CSV
    writer.writeheader()

    # Écrire les informations des hôtels dans le fichier CSV, classées par ville
    for city, hotels in grouped_data.items():
        for hotel in hotels:
            writer.writerow({
                'city': city,
                'hotel_name': hotel['hotel_name'],
                'hotel_address': hotel['hotel_address'],
                'latitude' : hotel['latitude'],
                'longitude' : hotel['longitude'],
                'hotel_rating': hotel['hotel_rating'],
                'hotel_description': hotel['hotel_description'],
                'hotel_url': hotel['hotel_url']
            })

In [37]:
hotels = pd.read_csv("data/best_cities_hotels.csv")
hotels

,city,hotel_name,hotel_address,latitude,longitude,hotel_rating,hotel_description,hotel_url
0,Nimes,Hôtel Le Pré Galoffre,"3115 Route De Générac, 30900 Nîmes, France",43.788582,4.353803,8.2,L’Hôtel Le Pré Galoffre bénéficie d’un emplace...,https://www.booking.com/hotel/fr/lepregaloffre...
1,Nimes,Mezza'Nîmes,"21 Rue de Beaucaire, 30000 Nîmes, France",43.838989,4.368417,9.3,L’hébergement 1 étoile Mezza'Nîmes se trouve à...,https://www.booking.com/hotel/fr/mezza-nimes.f...
2,Nimes,Logis NIMOTEL Hôtel Cosy,"152 Rue Claude Nicolas Ledoux, 30900 Nîmes, Fr...",43.815576,4.346604,7.9,Le Logis NIMOTEL Hôtel Cosy vous accueille à 1...,https://www.booking.com/hotel/fr/nimotel.fr.ht...
3,Nimes,Appartement des Carmes,"29 Rue Pierre Semard, 30000 Nîmes, France",43.839448,4.366918,8.3,L’hébergement Appartement des Carmes vous accu...,https://www.booking.com/hotel/fr/appartement-d...
4,Nimes,"LE COSY Joli F2 proche gare, Arénes avec terra...","37 Rue de Toulouse, 30000 Nîmes, France",43.828005,4.374962,9.5,"L'établissement LE COSY Joli F2 proche gare, A...",https://www.booking.com/hotel/fr/le-cosy-nimes...
...,...,...,...,...,...,...,...,...
120,Aigues Mortes,Résidence Odalys Le Mas des Flamants,"60 Chemin de Trouche, 30220 Aigues-Mortes, France",43.573794,4.180622,8.4,Située à l'extérieur des fortifications d'Aigu...,https://www.booking.com/hotel/fr/residence-le-...
121,Aigues Mortes,Les Jardins du Canal Piscine Chauffée,"440 Route De Nimes, 30220 Aigues-Mortes, France",43.572988,4.194825,8.8,L’hôtel Canal Aigues Mortes vous accueille sur...,https://www.booking.com/hotel/fr/canal-aigues-...
122,Aigues Mortes,Hôtel Le Médiéval,"221 Avenue Du Pont De Provence, 30220 Aigues-M...",43.571866,4.193662,8.8,L'Hôtel Le Médiéval est situé sur les rives du...,https://www.booking.com/hotel/fr/le-medieval.f...
123,Aigues Mortes,Hôtel Saint Louis,"10, Rue Amiral Courbet, 30220 Aigues-Mortes, F...",43.567220,4.189344,8.5,"Situé dans le cœur historique d'Aigues-Mortes,...",https://www.booking.com/hotel/fr/saint-louis-a...


In [38]:
#les nouveaux hotels n'ont pas encore de notation sur booking. Correction faite pour éviter les erreurs
hotels['hotel_rating'] = pd.to_numeric(hotels['hotel_rating'], errors='coerce')
hotels['hotel_rating'] = hotels['hotel_rating'].fillna("Nouvel établissement sur Booking. Pas encore de notation")

In [39]:
hotels["city"].value_counts()

city
Nimes                 25
Bormes les Mimosas    25
Marseille             25
Cassis                25
Aigues Mortes         25
Name: count, dtype: int64

In [40]:
#Rapide EDA pour identifier les valeurs min, max et la répartition des notes.
print("min:",hotels["hotel_rating"].min())
print("max:",hotels["hotel_rating"].max())
print(hotels["hotel_rating"].value_counts())

min: 6.3
max: 10.0
hotel_rating
8.9     11
9.3     11
8.6     10
9.6      9
8.3      7
8.8      6
9.5      6
9.1      6
9.2      6
8.2      5
8.7      5
8.4      4
9.4      4
8.5      4
9.0      4
8.1      4
7.7      4
8.0      3
9.7      3
7.8      2
7.5      2
7.4      2
7.9      2
9.9      2
7.3      1
10.0     1
6.3      1
Name: count, dtype: int64


In [55]:
from branca.element import Template, MacroElement

# Fonction pour convertir une note en couleur entre rouge (0) et vert (10)
def rating_to_color(rating, min_rating=6, max_rating=10):
    try:
        rating = float(rating)
        cmap = colormaps['RdYlGn']
        normalized = (rating - min_rating) / (max_rating - min_rating)
        rgba = cmap(normalized)
        return rgb2hex(rgba)
    except (ValueError, TypeError):
        return '#999999'  # Gris pour note absente / non numérique

# Créer une carte, initialisation vide
map = folium.Map(location=[0, 0], zoom_start=2)
# Liste pour stocker les coordonnées
bounds = []


# Markers avec couleurs dynamiques
for _, row in hotels.iterrows():
    lat, lon = row['latitude'], row['longitude']
    bounds.append([lat, lon])
    color = rating_to_color(row['hotel_rating'])

    folium.CircleMarker(
        location=[lat, lon],
        radius=7,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=folium.Popup(
            f"<b>{row['hotel_name']}</b><br>{row['hotel_address']}<br><b>Note:</b> {row['hotel_rating']}<br><br>{row['hotel_description']}<br><a href='{row['hotel_url']}' target='_blank'>Voir</a>",
            max_width=300
        )
    ).add_to(map)

# Centrer la carte sur tous les markers
map.fit_bounds(bounds)


legend_html = """
{% macro html(this, kwargs) %}

<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 200px; height: 90px; 
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
    ">
    <b>Hotel rating</b><br>
    <div style="
        background: linear-gradient(to right, red, yellow, green); 
        height: 15px; 
        margin-top: 5px;
        margin-bottom: 5px;
    "></div>
    <div style="display: flex; justify-content: space-between;">
        <span>6</span><span>8</span><span>10</span>
    </div>
</div>

{% endmacro %}
"""

legend = MacroElement()
legend._template = Template(legend_html)
map.get_root().add_child(legend)

map.save('maps/best_hotels.html')

# Afficher la carte
map


In [56]:
# Enregistrer le DataFrame en fichier CSV
hotels.to_csv('data/best_cities_hotels.csv', index=False)